In [ ]:
#import libraries
from splinter import Browser
from bs4 import BeautifulSoup
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo


In [ ]:
#import your web scraping file
import mission_to_mars 

In [ ]:
# create instance of Flask app
app = Flask(__name__)

# Use flask_pymongo to set up mongo connection ##NOT SURE if i'm creating mars_app here or if it already needs to be created in mongo
app.config["MONGO_URI"] = "mongodb://localhost:27017/mars_app"
mongo = PyMongo(app)

In [ ]:
# create route that renders index.html
@app.route("/")
def index():
    #find data
    results = mongo.db.collection.find()
    #return template and data
    return render_template("index.html", data=data)

In [ ]:
# route that will trigger scrape functions
@app.route("/scrape")
def scrape():
    # run scraped functions
    news = mission_to_mars.scrape_news()
    img_url = mission_to_mars.scrape_JPL()
    tables = mission_to_mars.scrape_facts()
    
    #store results in dictionary
    result = {
        "date": news["date"],
        "title": news["title"],
        "text": news["text"],
        "image": img_url["img_url"]
        "facts": tables["tables"]                     
    }
    
    # Insert result into database
    mongo.db.collection.insert_one(result)

    # Redirect back to home page
    return redirect("/", code=302)


if __name__ == "__main__":
    app.run(debug=True)  
